In [ ]:
!!pip install numpy

In [79]:
import torch
from torch.nn import functional as F
import torch.nn as nn
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
    print("Running on MPS Device")
else:
    print("MPS device not found")

tensor([1.], device='mps:0')
Running on MPS Device


In [80]:
max_iters = 5000
eval_iters = 200
eval_interval = 300
learning_rate = 1e-3
n_embed = 32

In [81]:
with open('data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [82]:
print("length of text file = ", len(text))

length of text file =  1115393


In [83]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [84]:
chars = sorted(list(set(text)))

In [85]:
vocab_size = len(chars)

In [86]:
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [87]:
# Character level language model
# Encode characters to numbers

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


# take a string output a list of integers
def encode(s): return [stoi[c] for c in s]
# decoder: take a list of integers, output a string
def decode(l): return ''.join([itos[i] for i in l])

In [88]:
print(encode("hii there"))

[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [89]:
print(decode(encode("hii there")))

hii there


In [90]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [91]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003853 111540


In [92]:
block_size = 8
train_data[: block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [93]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, output is {target}")

When input is tensor([18]), output is 47
When input is tensor([18, 47]), output is 56
When input is tensor([18, 47, 56]), output is 57
When input is tensor([18, 47, 56, 57]), output is 58
When input is tensor([18, 47, 56, 57, 58]), output is 1
When input is tensor([18, 47, 56, 57, 58,  1]), output is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), output is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), output is 58


In [94]:
print(x, y)

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [95]:
torch.randint(10, (5,))

tensor([7, 6, 8, 4, 1])

In [96]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batches(split):
    # generate a small batch of data of inputs and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(mps_device), y.to(mps_device)
    return x, y


xb, yb = get_batches('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]], device='mps:0')
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]], device='mps:0')
---
when input is [53], the target is: 59
when input is [53, 59], the target is: 6
when input is [53, 59, 6], the target is: 1
when input is [53, 59, 6, 1], the target is: 58
when input is [53, 59, 6, 1, 58], the target is: 56
when input is [53, 59, 6, 1, 58, 56], the target is: 47
when input is [53, 59, 6, 1, 58, 56, 47], the target is: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40], the target is: 59
when input is [49], the target is: 43
when input is [49, 43], the target is: 43
when input is [49, 43, 43], the target is: 54
when input is [49, 43, 43, 54], the target

In [97]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [98]:
class Head(nn.Module):
  """ One head of self-attention"""

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))


  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x) #(B,T,C)
    q = self.query(x) #(B,T,C)
    # compute self-attention scores ("affinities")
    wei = q @ k.transpose(-2, -1) * C**-0.5 # shouldn't it be head_size instead of C?
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    v = self.value(x)
    out = wei @ v
    return out

In [99]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # number of embedding dimensions n_embed
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)  # Language modeling head

    def forward(self, idx, targets=None):
        B,T = idx.shape

        tok_emb = self.token_embedding_table(idx)  # (batch, time, channel)
        pos_emb = self.position_embedding_table(torch.arange(T, device=mps_device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C) , broadcasting along the Batch dimension, for pos_emb, a new batch dim is added
        x = self.sa_head(x)
        logits = self.lm_head(x)  # (B,T,vocab_size) 

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T)  array of indices in the current context

        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [100]:
model = BigramLanguageModel()
m = model.to(mps_device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
context = torch.zeros((1, 1), dtype=torch.long, device=mps_device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))

ValueError: Expected input batch_size (4) to match target batch_size (32).

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f} val loss {losses['val']:.4f}")

    xb, yb = get_batches('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=mps_device)
print(decode(m.generate(idx=context, max_new_tokens=500)[0].tolist()))

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

### We want $x[b,t] = mean_{(i≤t)}\ x[b,i]$

In [ ]:

xbow = torch.zeros((B, T, C))  # bow = bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]  # (t,C)
        xbow[b, t] = torch.mean(xprev, 0)

### Same thing using Matrix multiplication

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
# a = torch.ones(3,3)
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a = ")
print(a)
print("--")
print("b = ")
print(b)
print("--")
print("c = ")
print(c)

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B,T,T)  @ (B,T,C) --> (B,T,C) pytorch adds this B dimension
torch.allclose(xbow, xbow2)  # same thing

#### Remember softmax: $σ(z)_i= \frac{e^{z_i}}{Σ^{K}_{j=1}e^{z_j}}$ for $i=1,...,K$ and $z=(z_1,...,z_K)\in\R^{K}$

In [ ]:
tril = torch.tril((torch.ones(T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x
print(wei)
# print(torch.allclose(xbow, out))
print(out.shape)

### Self attention head

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, H=16)
q = query(x) # (B, T, H=16)
wei = q @ k.transpose (-2, -1)  # (B,T,16) @ (B,16,T) --> (B,T,T) for each batch we get (T,T) tensor which are affinities

tril = torch.tril((torch.ones(T, T)))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
# out = wei @ x
out = wei @ v
# print(torch.allclose(xbow, out))
print(f'out: {out.shape}, wei: {wei.shape}, x: {x.shape}' )
print(wei[0])

In [ ]:
k = torch.randn(B,T, head_size)
q = torch.randn(B,T, head_size)
#wei = q @ k.transpose(-2,-1)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [ ]:
wei.var()